In [6]:
import itertools 

import numpy as np
from qiskit import *

# Introduction

My objective is to find the following Hamiltonian ground state. 

$$
H
=
\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & -1  & 0 \\ 0 & -1 & 0  & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$$

In other words, we want to figure out the eigenvector with the minimum eigenvalue. In order to solve this problem we'll use a **variational quantum eigensolver** (VQE).  

*With a simple 4x4 matrix it is easy to find the ground state with any numerical python library (ex: numpy). However, I will use this simple matrix to learn how to implement VQE and I will use numpy in order to verify, if VQE gets to the ground state.*

## Ground state using numpy

In [2]:
hamiltonian = np.array([[1,0,0,0],
                        [0,0,-1,0],
                        [0,-1,0,0],
                        [0,0,0,1]])

In [3]:
def compute_ground_state(matrix):
    eig_values, eig_vectors = np.linalg.eig(matrix)
    return np.min(eig_values), eig_vectors[:, np.argmin(eig_values)]

ground_energy, ground_state = compute_ground_state(hamiltonian)

# Reshape is used, just to visualize the ground state as a column vector.
print(f"Minimun energy = {ground_energy} with a state = \n{ground_state.reshape(-1, 1)}")

Minimun energy = -1.0 with a state = 
[[0.        ]
 [0.70710678]
 [0.70710678]
 [0.        ]]


Notice that the state $$[0, \frac{1}{\sqrt{2}}, \frac{1}{\sqrt{2}}, 0]$$ can be rewritten in the computational basis as: $$\frac{1}{\sqrt{2}}\big[ \left|01\right\rangle + \left|10\right\rangle \big]$$

## Variational Quantum Eigensolver (VQE)

VQE is an algorithm that finds the smallest eigenvalue and the associated eigenvector from a given Hamiltonian. The idea behind this algorithm is quite simple if we understand the **variational principle**.   

The variational principle consists in choosing a state $\left| \psi \right\rangle$. Then, we can compute the expected energy of the system as:  

$$ E =\left\langle \psi \right| H \left| \psi \right\rangle$$

So, our first guess, $\left| \psi \right\rangle$, has an expected energy $E$. Now, it would be really cool if in the next step we can do a second guess $\left| \psi_{2} \right\rangle$ with and expected energy $E_{2}$ lower than $E$, so $E_{2} < E$. Indeed this is variational prinicple objective's, we will have a first guess (also called ansatz) with a parameter $\theta$, such as $\left| \psi(\theta) \right\rangle$. Then, we will modify the parameter $\theta$ to get a new energy $E_{3}$. 

$$ \left\langle \psi(\theta_{i}) \right| H \left| \psi(\theta_{i}) \right\rangle = E(\theta_{i}) \geq E_{gs} $$

Trying all parameters will certainly get us to the ground state, but it is a naive approach since we will be wasting computational power in trying states $\left| \psi_{i} \right\rangle$ that would not lead us to our objective (finding the ground state). So, we need to tackle this problem with a better approach called VQE.

VQE it is a hybrid quantum-classical algorithm because given our guess $\left| \psi(\theta) \right\rangle$ it computes the energy of the system using a quantum circuit $E(\theta)$, and then it finds a better parameter $\theta$ using an optimization algorithm (Gradient Descent, Simultaneous Perturbation Stochastic Approximation, etc) run in a classical computer [[1]](https://qiskit.org/textbook/ch-applications/vqe-molecules.html).


Choosing a good ansatz it is crucial if we want to get the ground state, otherwise we can get stuck on a local minimum. It is important to assume that, we will **not** always find the best ansatz. In this cases, you can try different ansatzs and pick the one that works best. A general approach on how to try different ansatz can be found in qiskit documentation. For [2-qubits](https://qiskit.org/textbook/ch-applications/vqe-molecules.html#Simple-Variational-Forms) systems and for more general cases, let's say [n-qubits](https://qiskit.org/textbook/ch-applications/vqe-molecules.html#Structure-of-Common-Variational-Forms)

**Why VQE instead of a classical eigensolver?**   

In physics to describe a quantum system, we use a Hamiltonian. This Hamiltonian is represented by a matrix that scales with the size of the quantum system. Specifically with a n-qubit system the matrix will be $2^{n}x2^{n}$, so it scales exponentially. Using a classical eigensolver for a 50-qubit system is not doable on any current or future processor, however it will be a solvable by a quantum computer.

### 1st step: Decomposition
As I said previously, VQE computes the energy of the system using a quantum circuit. All the information about the system is compressed in the Hamiltonian matrix ($H$). So, what we need to do is rewrite the Hamiltonian matrix in new terms (new operators) so that expectations values can be measured using the quantum computer. 

A quantum computer measures states in the computational basis: $\left| 0\right\rangle, \left| 1\right\rangle$. And the operator that act on this basis is the well known $\sigma_{z}$. So it should come as no surprise, then, that it will be a good idea to rewrite the previous Hamiltonian in terms of the Pauli's matrices: $ I, \sigma_{z}, \sigma_{x}, \sigma_{y}$.

$$
I = \begin{pmatrix}
1 & 0\\
0 & 1
\end{pmatrix},
\qquad
\sigma_{z} = \begin{pmatrix}
1 & 0\\
0 & -1
\end{pmatrix},
\qquad
\sigma_{x} = \begin{pmatrix}
0 & 1\\
1 & 0
\end{pmatrix},
\qquad
\sigma_{y} = \begin{pmatrix}
0 & -i\\
i & 0
\end{pmatrix}.
$$

*To get a better understading on what the Pauli's matrices represent I would recommend reading Chapter 3 of **Quantum mechanics The Theoretical Minimum** by Leonard Susskind & Art Friedman.*

This 4 matrices form a basis, that span the space of a 2-dimensional Hilbert space. The problem is that our matrix is in a 4-dimensioanal Hilbert space (4x4 Hamiltonian), hence we will decompose $H$ into the sum of **tensor products** of Pauli matrices, multiplied by some constant coefficients.

Now, it is where the fun beginings because we need to figure out how to represent $H$ in terms of $ I, \sigma_{z}, \sigma_{x}, \sigma_{y}$.  

If we see 1's in the diagonal we should think about using the tensor product: $I\otimes I $.

$$
H=\begin{pmatrix} 
\bf{1} & 0 & 0 & 0 \\ 
0 & 0 & -1  & 0 \\ 
0 & -1 & 0  & 0 \\ 
0 & 0 & 0 & \bf{1} 
\end{pmatrix}
\qquad
I\otimes I = \begin{pmatrix} 
\bf{1} & 0 & 0 & 0 \\ 
0 & 1 & 0 & 0 \\ 
0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & \bf{1} 
\end{pmatrix}
$$

Things are going well, with only one tensor product we have been able to represent numbers 2 of $H$. Now to figure out the final decomposition of the Hamiltonian I would recommend having a paper a pencil and write down all the combinations of Pauli's matrices (there are a total 20 combinations: $I\otimes I, \sigma_z \otimes \sigma_z,\sigma_z \otimes \sigma_x, etc$. Since tensor product is not commutative). Trust me, once you have written down the 20 combinations there aren't that many useful tensor products (combinations of Pauli matrix) to solve the puzzle.  

However, if in the future you are interested in another hamiltonian you will have to compute again all 20 combinations and figure out which combinations of Pauli matrices represent your new hamiltonian. For this reason, it is a good practice to write a code that is able to find how to decompose your hamiltonian into Pauli matrices. _The following code it is a generalization for any hamiltonian of size $2^{n} \times 2^{n}$, so in the following section I have written down the concepts on how to decompose a hamiltonian into a sum of the Pauli matrix tensor products, for the interested reader_

In [4]:
pauli_matrices = {
    'sigma_x' : np.array([[0, 1],  [ 1, 0]], dtype=np.complex128),
    'sigma_y' : np.array([[0, -1j],[1j, 0]], dtype=np.complex128),
    'sigma_z' : np.array([[1, 0],  [0, -1]], dtype=np.complex128),
    'identity' : np.array([[1, 0],  [ 0, 1]], dtype=np.complex128)
}

## TODO: check for other hamiltonian and in higher dimensions.
def decompose(hamiltonian):
    
    # Input hamiltonian must obey these properties.
    x_dim, num_matrices = check_properties(hamiltonian)
    
    decomposition = str()
    matrices_span = list(itertools.product(pauli_matrices, repeat=int(num_matrices))) 
    
    for element in matrices_span:
        basis = pauli_matrices[element[0]] if x_dim == 2 else tensor_product(element)
        
        # Tensor products of pauli matrices must be normalized (ex: A = X \otimes X, Tr(AA)=4)
        normalization_factor = 1/x_dim
        component = normalization_factor * hilbert_schmidt_product(basis, hamiltonian)
        if component != 0:
            decomposition += (str(component) +' '+ print_operators(element))
            decomposition += '\n'
    
    return decomposition
            

def hilbert_schmidt_product(matrix_1, matrix_2):
    ''' 
    H-S product is defined as the trace of A^{\dagger} times B
    in this case matrix A is hermitian so H-S product can be 
    rewritten as trace of A times B.
    '''
    return np.matmul(matrix_1, matrix_2).trace()


def tensor_product(matrices):
    ''' 
    Recursive function to compute the tensor product of n-pauli matrices
    Ex: - \sigma_x \otimes \sigma_y
        - \sigma_z \otimes \sigma_z \otimes \sigma_z
    '''
    matrix_1 = pauli_matrices[matrices[0]] if isinstance(matrices[0], str) else matrices[0]
    matrix_2 = pauli_matrices[matrices[1]]

    if len(matrices) == 2:
        return np.kron(matrix_1, matrix_2)
    else:
        new_operator = np.kron(matrix_1, matrix_2)

        matrices = (new_operator, *matrices[2:])
        return tensor_product(matrices)

    
def is_power_of_two(n): 
    if (n == 0): 
        return False
    while (n != 1): 
            if (n % 2 != 0): 
                return False
            n = n // 2  
    return True


def print_operators(operators):
    renaming_op = {'sigma_x': 'X', 'sigma_y': 'Y', 'sigma_z': 'Z', 'identity': 'I'}
    chain = str()
    for operator in operators:
        chain += (renaming_op[operator] + ' \otimes ')
    return ' '.join(chain.split(' ')[:-2]) 


def check_properties(hamiltonian):
    x_dim, y_dim = hamiltonian.shape
    if x_dim != y_dim:
        raise ValueError("Hamiltonian must be an square matrix with shape nxn") 
    hermitian_h = hamiltonian.transpose().conjugate()
    
    if not np.allclose(hermitian_h, hamiltonian):
        raise ValueError("Hamiltonian is not hermitian")
        
    num_matrices = np.log2(x_dim)
    if not is_power_of_two(num_matrices):
        raise NotImplementedError("Dimension should be a power 2. Generalizations of Pauli matrices for other dimensions is not implemented")
    
    return x_dim, num_matrices

In [5]:
print(decompose(hamiltonian))

(-0.5+0j) X \otimes X
(-0.5+0j) Y \otimes Y
(0.5+0j) Z \otimes Z
(0.5+0j) I \otimes I



So our hamiltonian can be written as: 

$$ H = \frac{I+\sigma_{z}}{2} - \frac{\sigma_{x}+ \sigma_{y}}{2}$$

**Details about Hamiltonian decomposition.**

First of all, it is important to know how to decompose a vector $\vec{v}$ in terms of a orthogonal basis (ex: standard basis). If you need to refresh this concept I would recommend this [video](https://www.youtube.com/watch?v=joa11IB4maQ&ab_channel=MathTheBeautiful).

Now, as I said before $\{ I, \sigma_{z}, \sigma_{x}, \sigma_{y}\}$ is a basis that span a 2-dimensional hilbert spaces. Then, as we already know a hamiltonian that represent a quantum system also lives in a hilbert space, hence if our hamiltonian is a $2 \times 2$ matrix 

$$ H = \begin{bmatrix} 5 & 1 \\ 1 & -1 \end{bmatrix} $$

can be rewritten in terms of the previuos basis   

$$ H = \alpha I + \beta \sigma_{z} + \gamma \sigma_{x} + \delta \sigma_{y}. $$

To figure out the components $(\alpha, \beta, \gamma, \delta)$, we also the same trick from the previous [video](https://www.youtube.com/watch?v=joa11IB4maQ&ab_channel=MathTheBeautiful) called **inner product**. Nevertheless, we are not dealing with a ordinary orthogonal basis (vector), but we are dealing with matrices. So, we need a generalization of the inner product for these matrices called **Hilbert-Schmidt product**.  

$$ \langle A, B \rangle = \mathrm{Tr}(A^{\dagger}B) $$

Next, step is to find out the components. Remember, Pauli matrices are hermitian, so $\sigma_{z}^{\dagger} = \sigma_z$.

$$\mathrm{Tr}(\sigma_{z}H)=\alpha \mathrm{Tr}(\sigma_{z}I)+\beta\mathrm{Tr}(\sigma_{z}\sigma_{z})+\gamma\mathrm{Tr}(\sigma_{z}\sigma_{x})+\delta\mathrm{Tr}(\sigma_{z}\sigma_{y}) $$

All terms are traceless, except $\mathrm{Tr}(\sigma_{z}\sigma_{z})=6$.   

$$\mathrm{Tr}(\sigma_{z}H)= 6$$
$$\alpha \mathrm{Tr}(\sigma_{z}I)+\beta\mathrm{Tr}(\sigma_{z}\sigma_{z})+\gamma\mathrm{Tr}(\sigma_{z}\sigma_{x})+\delta\mathrm{Tr}(\sigma_{z}\sigma_{y})=2\beta$$

Then $\beta=3$. So, I implented this method in a python function called **decompose**: 

```python
H_2x2 = np.array([[5,1],[1, -1]])
print(decompose(H_2x2))
```

and we get 

$$ H_{2\times 2} = \sigma_{x} + 3\sigma_{z} + 2I $$

But, our current Hamiltonian belongs to 4-dimensional Hilbert-space, is it possible to generalize this method? Luckly, yes. However the complexity increase exponentially, because in order to represent a 4-dimensional Hilbert-space we compute all combinations of a tensor products between two Pauli matrices. Such that the hamiltonian is written as

$$ H = 
\alpha_1 \sigma_x \otimes \sigma_x + \alpha_2 \sigma_x \otimes \sigma_y + \alpha_3 \sigma_x \otimes \sigma_z + \alpha_4 \sigma_x \otimes I +\\ 
\alpha_5 \sigma_y \otimes \sigma_y + \alpha_6 \sigma_y \otimes \sigma_x + \alpha_7 \sigma_y \otimes \sigma_z + \alpha_8 \sigma_y \otimes I +\\ 
\alpha_9 \sigma_z \otimes \sigma_z + \alpha_{10} \sigma_z \otimes \sigma_x + \alpha_{11} \sigma_z \otimes \sigma_y + \alpha_{12} \sigma_z \otimes I +\\ 
\alpha_{13} I \otimes I + \alpha_{14} I \otimes \sigma_x + \alpha_{15} I \otimes \sigma_y + \alpha_{16}  I \otimes \sigma_z \\ 
$$

Using the function **decompose**, we will get all the non-zero components ($a_i$) and get the Hamiltonian rewritten into as the sum of the Pauli matrix tensor products.


**references:**  
https://quantumcomputing.stackexchange.com/questions/6882/decomposition-of-a-matrix-in-the-pauli-basis
https://quantumcomputing.stackexchange.com/questions/8725/can-arbitrary-matrices-be-decomposed-using-the-pauli-basis
https://michaelgoerz.net/notes/decomposing-two-qubit-hamiltonians-into-pauli-matrices.html

### 2nd Step: Ansatz

Choosing a good ansatz is complicate because you need to balance two opposite things.
1. Construct a state $ \left| \psi \right\rangle $ that is able to generate all or almost all the states in the Hilbert space.
2. Construct a quantum circuit with as few parameters as possible. 

As you may expect an ansatz with a few parameters will not visit the entire Hilbert space and ansatz that is able to visit the entire Hilbert space will have a lot of parameters.  However, in a 4-dimensional Hilbert space we can implement both ansatz. In higher Hilbert space option 1 it is impossible to implement for a VQE algorithm because the classical optimizer is not able to handle an exponential number of parameters as the system grows.

In this case, I will initialize the circuit with the best ansatz:

$$ (R_y \otimes I) C_{NOT} (H \otimes I) \left| 0 \right\rangle \otimes \left| 0 \right\rangle $$

Why is it the best ansatz? Because using numpy I've already computed the ground state and I know in which places of the Hilbert space to look for. I already know that the ground state is: 

$$\frac{1}{\sqrt{2}}\big[ \left|01\right\rangle + \left|10\right\rangle \big]$$

So I need to build a circuit that will provide this step. It is important to notice that, this state is called [Bell state](https://en.wikipedia.org/wiki/Bell_state) and it is really famous because it is a state of maximum entanglement between 2 qubits. Additionaly, it is related with a famous paradox called [EPR paradox](https://en.wikipedia.org/wiki/EPR_paradox).

Due to I knew it was a Bell state, I just need to Google: how to implement a Bell state? The answer is using a Hadamard gate and then a Control-Not gate.

<img src="images/Bell_state_00.png" alt="drawing"/>

So using this circuit our initial state $\left|00\right\rangle$ becomes

$$\frac{1}{\sqrt{2}}\big[ \left|00\right\rangle + \left|11\right\rangle \big]$$

Lastly, we need to apply a rotation in the x-axis or in the y-axis to get the ground state, in my case I would use a $R_y$ gate ($R_x$ would also works). So, my variational parameter will be $\theta$ (rotation angle on the y-axis). After several iterations on VQE, the classical optimizer will find out which is the value of $\theta$ that outputs this state:
$$\frac{1}{\sqrt{2}}\big[ \left|01\right\rangle + \left|10\right\rangle \big]$$.




In [7]:
def ansatz(circuit, theta):
    q = circuit.qregs[0]    
    circuit.h(q[0])
    circuit.cx(q[0], q[1])
    circuit.ry(theta, 0)
    return circuit

### 3rd step: Measurements

### References

1. [Qiskit documentation](https://qiskit.org/textbook/ch-applications/vqe-molecules.html)

Bonus:
    1. Utiliza un noise simulator.
    1. escriure circuit amb ansatz generica per hamiltonain 4x4
    2. Lletgir es circuits de es paper: 
    3. Implentar un VQE per hamiltoniano 8x8, provant ansatz calculada numericament.